# IMDB 영화평 감성분석(이진분류)

In [1]:
import numpy as np
import pandas as pd
from google.colab import files
up = files.upload()

Saving labeledTrainData.tsv to labeledTrainData.tsv


In [2]:
df = pd.read_csv('labeledTrainData.tsv', sep='\t')
df.head(3)

,id,sentiment,review
0,5814_8,1,With all this stuff going down at the moment w...
1,2381_9,1,"\The Classic War of the Worlds\"" by Timothy Hi..."
2,7759_3,0,The film starts with a manager (Nicholas Bell)...


In [3]:
df = pd.read_csv('labeledTrainData.tsv', sep='\t', quoting=3) # 3: QUOTE_NONE
df.head(3)

,id,sentiment,review
0,"""5814_8""",1,"""With all this stuff going down at the moment ..."
1,"""2381_9""",1,"""\""The Classic War of the Worlds\"" by Timothy ..."
2,"""7759_3""",0,"""The film starts with a manager (Nicholas Bell..."


In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 25000 entries, 0 to 24999
Data columns (total 3 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   id         25000 non-null  object
 1   sentiment  25000 non-null  int64 
 2   review     25000 non-null  object
dtypes: int64(1), object(2)
memory usage: 586.1+ KB


In [5]:
df.review[0][:1000]

'"With all this stuff going down at the moment with MJ i\'ve started listening to his music, watching the odd documentary here and there, watched The Wiz and watched Moonwalker again. Maybe i just want to get a certain insight into this guy who i thought was really cool in the eighties just to maybe make up my mind whether he is guilty or innocent. Moonwalker is part biography, part feature film which i remember going to see at the cinema when it was originally released. Some of it has subtle messages about MJ\'s feeling towards the press and also the obvious message of drugs are bad m\'kay.<br /><br />Visually impressive but of course this is all about Michael Jackson so unless you remotely like MJ in anyway then you are going to hate this and find it boring. Some may call MJ an egotist for consenting to the making of this movie BUT MJ and most of his fans would say that he made it for the fans which if true is really nice of him.<br /><br />The actual feature film bit when it finally

- 텍스트 전처리

In [11]:
# <br /> 태그는 공백으로 변환
df.review = df.review.str.replace('<br />', ' ')

In [12]:
# 구둣점, 숫자 제거 - 영문자가 아닌 글자는 공백으로 변환
# 데이터프레임의 str method는 정규표현식을 지원함
df.review = df.review.str.replace('[^A-Za-z]', ' ').str.strip()
df.review[0][:1000]

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: FutureWarning: The default value of regex will change from True to False in a future version.
  


'With all this stuff going down at the moment with MJ i ve started listening to his music  watching the odd documentary here and there  watched The Wiz and watched Moonwalker again  Maybe i just want to get a certain insight into this guy who i thought was really cool in the eighties just to maybe make up my mind whether he is guilty or innocent  Moonwalker is part biography  part feature film which i remember going to see at the cinema when it was originally released  Some of it has subtle messages about MJ s feeling towards the press and also the obvious message of drugs are bad m kay  br    br   Visually impressive but of course this is all about Michael Jackson so unless you remotely like MJ in anyway then you are going to hate this and find it boring  Some may call MJ an egotist for consenting to the making of this movie BUT MJ and most of his fans would say that he made it for the fans which if true is really nice of him  br    br   The actual feature film bit when it finally sta

- Train/Test dataset 분리

In [30]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(
    df.review, df.sentiment, stratify=df.sentiment, random_state=2022
)
y_train.value_counts()

0    9375
1    9375
Name: sentiment, dtype: int64

- CountVectorizer로 변환

In [14]:
from sklearn.feature_extraction.text import CountVectorizer
cvect = CountVectorizer(stop_words='english')

In [31]:
cvect.fit(X_tain)
X_train_cv = cvect.transform(X_train)
X_train_cv.shape

(18750, 65213)

In [16]:
# test dataset은 train dataset을 변환한 Vectorizer로 변환해야 함
X_test_cv = cvect.transform(X_test)
X_test_cv.shape

(6250, 65213)

- 분류기: Naive Bayes

In [26]:
from sklearn.naive_bayes import MultinomialNB
nb = MultinomialNB()

In [27]:
% time nb.fit(X_train_cv, y_train)

CPU times: user 25.2 ms, sys: 2.01 ms, total: 27.2 ms
Wall time: 28.4 ms


MultinomialNB()

In [28]:
nb.score(X_test_cv, y_test)

0.85536

- ngram_range = (1,2)

In [40]:
cvect2 = CountVectorizer(ngram_range=(1,2), stop_words='english')
cvect2.fit(X_train)
X_train_cv2 = cvect2.transform(X_train)
print(X_train_cv2.shape)

(18750, 1377719)


In [41]:
X_test_cv2 = cvect2.transform(X_test)
X_test_cv2.shape

(6250, 1377719)

In [44]:
nb2 = MultinomialNB()
nb2.fit(X_train_cv2, y_train)

MultinomialNB()

In [45]:
nb2.score(X_test_cv2, y_test)

0.87104

- 모델 저장하고 불러오기

In [42]:
import joblib

In [46]:
joblib.dump(cvect2, 'imdb_cvect12.pkl')
joblib.dump(nb2, 'imdb_nb2.pkl')

['imdb_nb2.pkl']

In [47]:
!ls -l

total 113792
-rw-r--r-- 1 root root 38869270 Apr 28 02:40 imdb_cvect12.pkl
-rw-r--r-- 1 root root 44087725 Apr 28 02:40 imdb_nb2.pkl
-rw-r--r-- 1 root root 33556378 Apr 28 01:35 labeledTrainData.tsv
drwxr-xr-x 1 root root     4096 Apr 25 13:46 sample_data


In [48]:
new_cvect = joblib.load('imdb_cvect12.pkl')
new_nb = joblib.load('imdb_nb2.pkl')

- 실제 데이터로 검증

In [49]:
review = '''
Robert P is a terrible Batman. Absolutely no Bat-charisma at all. 
The Riddler character is a complete bore! I hated all of this film. 
I certainly do not understand the good reviews. 
It's not close to the Dark Knight caliber. Please do not make a sequel.
'''

In [51]:
# 텍스트 전처리
import re
review = re.sub('[^A-Za-z]', ' ', review).strip()

In [53]:
# feature 변환
review_cv = new_cvect.transform([review])
review_cv.shape

(1, 1377719)

In [54]:
new_nb.predict(review_cv)

array([0])